In [65]:
import numpy as np
import cv2
from pycocotools.coco import COCO

# Ruta al archivo de anotaciones COCO
ann_file_test = './Dataset/Segundoparcialv7icoco-mmdetection/test/_annotations.coco.json'
ann_file_train = './Dataset/Segundoparcialv7icoco-mmdetection/train/_annotations.coco.json'
ann_file_val = './Dataset/Segundoparcialv7icoco-mmdetection/valid/_annotations.coco.json'

# Inicializar el objeto COCO con el archivo de anotaciones
coco_test = COCO(ann_file_test)
coco_train = COCO(ann_file_train)
coco_val = COCO(ann_file_val)
#print("-------------------")
#print(coco.info())
#print("-------------------")

# Obtener los identificadores de las imágenes en el conjunto de datos
img_ids_test = coco_test.getImgIds()
print("Cant. images test: ", len(img_ids_test))
img_ids_train = coco_train.getImgIds()
print("Cant. images train: ", len(img_ids_train))
img_ids_val = coco_val.getImgIds()
print("Cant. images valid: ", len(img_ids_val))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Cant. images test:  47
Cant. images train:  717
Cant. images valid:  48


In [68]:
def create_mask(coco, img_id, num_classes):
    img_info = coco.loadImgs(img_id)[0]
    ann_ids = coco.getAnnIds(imgIds=img_id)
    annotations = coco.loadAnns(ann_ids)
    a = img_info['file_name'].split('.')[0]
    b = img_info['file_name'].split('.')[1]
    c = img_info['file_name'].split('.')[2]
    name_mask = a + '.' + b + '.' + c + '.png'
    
    # Crear una lista de máscaras para cada clase
    masks = [np.zeros((img_info['height'], img_info['width']), dtype=np.uint8) for _ in range(num_classes)]
    
    # Rellenar las máscaras con las regiones de interés de las anotaciones COCO
    for annotation in annotations:
        segmentations = annotation['segmentation']
        class_id = annotation['category_id'] - 1  # Identificador de clase
        
        for segmentation in segmentations:
            # Desempaquetar las tuplas de las coordenadas de segmentación
            points = np.array(segmentation, dtype=np.int32).reshape((-1, 2))
            
            # Convertir la segmentación en un polígono y asignar el valor de clase en la máscara
            cv2.fillPoly(masks[class_id], [points], 1) # type: ignore
    
    # Concatenar las máscaras en un solo array tridimensional
    mask = np.stack(masks, axis=-1)
    
    return mask, name_mask

In [74]:
def generate_masks(num_classes):
    for img_id in img_ids_test:
        mask, name_mask = create_mask(coco_test, img_id, num_classes)
        cv2.imwrite('./Dataset/Segundoparcialv7icoco-mmdetection/testmasks/' + name_mask, mask*255)
    for img_id in img_ids_train:
        mask, name_mask = create_mask(coco_train, img_id, num_classes)
        cv2.imwrite('./Dataset/Segundoparcialv7icoco-mmdetection/trainmasks/' + name_mask, mask*255)
    for img_id in img_ids_val:
        mask, name_mask = create_mask(coco_val, img_id, num_classes)
        cv2.imwrite('./Dataset/Segundoparcialv7icoco-mmdetection/validmasks/' + name_mask, mask*255)

In [75]:
generate_masks(num_classes=3)
print("Done")

Done
